In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
df = pd.read_csv('features.csv')

In [3]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [4]:
df_train = pd.read_json('./train.json/train.json',lines=True)

In [5]:
df_train = df_train[df_train['summary'].notna()]

In [6]:
len(df_train)

199965

In [7]:
df_train = df_train[:len(df)]

In [9]:
df.reset_index(drop=True, inplace=True)
df_train.reset_index(drop=True, inplace=True)

In [30]:
dataset = pd.concat([df, df_train[['category','price','overall','summary','reviewerID','itemID']]], axis=1) 

In [31]:
len(dataset)

199000

In [32]:
dummy = pd.get_dummies(dataset['category'], drop_first=True)

In [33]:
len(dummy)

199000

In [34]:
dataset = pd.concat([dataset,dummy], axis=1) 

In [36]:
dataset = dataset[dataset['price'].str.startswith('$')]

In [37]:
dataset['price'] = dataset[dataset['price'].str.startswith('$')].price.str[1:].astype(float)

In [38]:
dataset.reset_index(drop=True, inplace=True)

In [19]:
X = dataset.drop(columns=['overall','summary','category','reviewerID','itemID'])

In [20]:
y = dataset['overall']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1,shuffle=False)

In [22]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,763,764,765,766,767,price,Classical,Dance & Electronic,Jazz,Pop
0,0.161135,0.000407,-0.068433,-0.129362,-0.184148,-0.432141,0.255694,0.536445,-0.085235,-0.129654,...,0.015972,-0.018755,-0.116121,0.540379,0.267593,35.93,0,0,0,1
1,-0.150302,-0.159166,0.141929,-0.082894,-0.123089,-0.091947,0.099853,0.174497,-0.118698,-0.273077,...,0.039808,0.008492,-0.016914,0.196060,0.085627,11.28,0,0,0,0
2,0.026183,-0.100280,-0.002925,-0.088067,-0.128211,-0.096697,0.345383,0.418556,-0.041921,-0.166147,...,0.016740,0.024604,-0.037092,0.304590,0.153733,89.86,0,0,0,1
3,-0.275129,-0.126827,-0.015942,-0.290908,-0.078654,-0.179946,0.089550,0.259193,-0.235870,-0.367499,...,-0.030864,-0.030396,-0.081538,0.234779,0.198854,11.89,0,0,0,1
4,0.068014,-0.031137,-0.101309,-0.090469,-0.007553,-0.336420,0.208208,0.486158,-0.091043,-0.315945,...,0.118615,-0.270578,-0.112419,0.363581,0.132633,15.24,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176653,-0.163558,-0.177608,0.007300,-0.146453,0.002007,-0.225201,0.184601,0.464533,-0.138601,-0.305808,...,-0.025221,-0.062820,-0.101833,0.296814,0.049899,16.98,0,0,1,0
176654,0.142340,-0.156731,0.200659,-0.062521,-0.322141,-0.235883,0.316947,0.179272,-0.059017,0.018997,...,0.047079,-0.032847,-0.116914,0.206815,0.133285,39.99,0,1,0,0
176655,0.038182,0.028659,0.107073,-0.175235,0.031803,-0.096688,0.198478,0.305384,-0.092591,-0.231175,...,-0.059083,-0.122240,-0.006917,0.208703,0.314089,0.97,0,0,0,1
176656,-0.276395,-0.227720,-0.109515,-0.106832,-0.133490,-0.023411,0.165993,0.104826,-0.132380,-0.207726,...,-0.088806,-0.086599,-0.095656,0.215206,0.288115,13.78,0,0,0,1


In [23]:
from sklearn.ensemble import RandomForestRegressor

In [24]:
regr = RandomForestRegressor(n_estimators=100, n_jobs=-1)

In [25]:
regr.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1)

In [26]:
from sklearn.metrics import mean_squared_error
mean_squared_error(regr.predict(X_test), y_test)

0.6631432891548252

In [28]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=100)
gbr.fit(X_train, y_train)

GradientBoostingRegressor()

In [29]:
mean_squared_error(gbr.predict(X_test), y_test)

0.6657552923355368

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
import torch
import torch.nn as nn
import pandas as pd
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 1, 
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
print('')
model.eval()
model.load_state_dict(torch.load('weight_2_best.pth'))
df_test = dataset.iloc[176658:]

In [ ]:
df_test['summary'] = df_test['category'] +', '+ df_test['summary']

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
summary_test = df_test['summary'].values

input_ids = []
attention_masks = []
for sent in summary_test:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, 
                        max_length = 79 ,           
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',
                        truncation=True
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

In [42]:
test_label=[]
for in_ids, mask, in zip(input_ids, attention_masks):
    in_ids = in_ids.to(device)
    mask = mask.to(device)
    logits_str = model(in_ids.unsqueeze(0), token_type_ids=None, attention_mask=mask.unsqueeze(0))[0]
    test_label.append(logits_str.detach().cpu().numpy()[0][0])

In [43]:
test_label = np.asarray(test_label).reshape(-1) + 1
test_label[test_label>5] = 5
test_label[test_label<1] = 1

In [44]:
mean_squared_error(test_label,y_test)

0.48417880333057856

In [45]:
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise.model_selection import GridSearchCV
from surprise import KNNWithMeans
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import SVDpp

In [46]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_train[["reviewerID", "itemID", "overall"]], reader)
trainset_cf, testset_cf = train_test_split(data, test_size=.1,shuffle=False)
svd = SVDpp(n_epochs=50,lr_all=0.007,reg_all=0.1)
svd.fit(trainset_cf)
df_test = dataset.iloc[176658:]
data_test = Dataset.load_from_df(df_test[["reviewerID", "itemID", "overall"]], reader)
trainset_cf1, testset_cf1 = train_test_split(data_test, test_size=1.0,shuffle=False)
predictions_svd = svd.test(testset_cf1)

In [49]:
prediction_svd = np.asarray(predictions_svd)[:,3]
mean_squared_error(prediction_svd,y_test)

0.8228861132473816

In [50]:
prediction_regr = regr.predict(X_test)
prediction_gbr = gbr.predict(X_test)
# prediction_svd
# test_label
# y_test

In [51]:
res_df = pd.DataFrame(
    {'regr': prediction_regr,
     'gbr': prediction_gbr,
     'svd': prediction_svd,
     'bert': test_label,
     'label': y_test
    })

In [53]:
#res_df.to_csv('results.csv',index=False)

In [ ]:
df1 = pd.read_csv('features_test.csv')
df1.drop(columns=['Unnamed: 0'],inplace=True)
df_train1 = pd.read_json('./test.json/test.json',lines=True)
len(df_train1)
df1.reset_index(drop=True, inplace=True)
df_train1.reset_index(drop=True, inplace=True)
dataset1 = pd.concat([df1, df_train1[['category','price']]], axis=1)
dummy1 = pd.get_dummies(dataset1['category'], drop_first=True)
dataset1 = pd.concat([dataset1,dummy1], axis=1) 
dataset1.drop(columns=['category'],inplace=True)
dataset1 = dataset1[dataset1['price'].str.startswith('$')]
dataset1['price'] = dataset1[dataset1['price'].str.startswith('$')].price.str[1:].astype(float)

In [56]:
predications = gbr.predict(dataset1)

In [59]:
predications[predications>5] = 5
predications[predications<1] = 1

In [62]:
index = 0
predictions = open('rating_predictions_gbr.csv', 'w')
for l in open('rating_pairs.csv'):
    if l.startswith('userID'):
        #header
        predictions.write(l)
        continue
    u,p = l.strip().split('-')
    predictions.write(u + '-' + p + ',' + str(predications[index]) + '\n')
    index += 1

In [ ]:
index

In [28]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm

In [29]:
class myDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        x = self.x.iloc[index].values
        y = self.y.iloc[index]
        return x,y


In [30]:
trainset = myDataset(X_train,y_train)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

In [31]:
testset = myDataset(X_test,y_test)
testloader = DataLoader(testset, batch_size=64, shuffle=True)

In [33]:
class Net(nn.Module):
    def __init__(self,n_input,n_output):
        super(Net,self).__init__()
        self.predict = nn.Linear(n_input,n_output)
    def forward(self,input):
        out =self.predict(input)

        return out

In [34]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = Net(773,1).to(device)
#optimizer = optim.Adam(net.parameters(),0.01)
loss_function = torch.nn.MSELoss()

In [36]:
optimizer = AdamW(net.parameters(),
                  lr = 2e-3, 
                  eps = 1e-8
                )

In [37]:
num_epoch = 10
total_steps = len(trainloader) * num_epoch

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

for epoch in range(num_epoch):
    print("start training")
    net.train()
    total_train_loss = 0
    for feature, label in tqdm(trainloader):
        feature = feature.float().to(device)
        label = label.float().to(device)
        prediction = net(feature)
        loss = loss_function(prediction, label.view(-1,1))
        total_train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
    print("Epoch: {}/{}".format(epoch+1, num_epoch),
        "Training Loss: {:.3f}".format(total_train_loss/len(trainloader)))
    
    print("")
    print("Running Validation...")
    
    net.eval()
    with torch.no_grad():
        total_eval_loss = 0
        for feature, label in tqdm(testloader):
            feature = feature.float().to(device)
            label = label.float().to(device)
            prediction = net(feature)
            loss = loss_function(prediction, label.view(-1,1))
            total_eval_loss += loss.item()
        print("Epoch: {}/{}".format(epoch+1, num_epoch),
        "Test Loss: {:.3f}".format(total_eval_loss/len(testloader)))
        
    
        

start training


100%|██████████████████████████████████████████████████████████████████████████████| 2761/2761 [01:39<00:00, 27.63it/s]


Epoch: 1/10 Training Loss: 0.824

Running Validation...


100%|████████████████████████████████████████████████████████████████████████████████| 307/307 [00:10<00:00, 30.03it/s]


Epoch: 1/10 Test Loss: 0.713
start training


...

Epoch: 9/10 Test Loss: 0.672
start training


100%|██████████████████████████████████████████████████████████████████████████████| 2761/2761 [01:38<00:00, 28.01it/s]


Epoch: 10/10 Training Loss: 0.675

Running Validation...


100%|████████████████████████████████████████████████████████████████████████████████| 307/307 [00:09<00:00, 33.85it/s]


Epoch: 10/10 Test Loss: 0.668
